In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load preprocessed time series data
df = pd.read_csv("../data/ts_data.csv", parse_dates=["pickup_hour"])

# Confirm structure
df.head()


,pickup_hour,4074.14,4298.05,4461.07,4762.05,4977.03,4993.02,4993.15,5024.10,5033.01,...,JC102,JC103,JC104,JC105,JC107,JC108,JC109,JC110,JC115,JC116
0,2024-01-01 00:00:00,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,2,0,3,11
1,2024-01-01 01:00:00,0,0,0,0,0,0,0,0,0,...,4,0,2,5,0,0,4,1,4,2
2,2024-01-01 02:00:00,0,0,0,0,0,0,0,0,0,...,2,2,4,1,0,0,2,1,6,4
3,2024-01-01 03:00:00,0,0,0,0,0,0,0,0,0,...,0,2,2,1,0,0,1,2,2,0
4,2024-01-01 04:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,4,0


In [13]:
# Select target location for baseline (e.g., HB102 or any location column)
target_location = "HB102"

# Create a copy with just pickup_hour and target location
df_baseline = df[["pickup_hour", target_location]].copy()

# Create 1-hour lag feature
df_baseline["lag_1"] = df_baseline[target_location].shift(1)

# Drop rows with NaNs caused by shifting
df_baseline = df_baseline.dropna().reset_index(drop=True)

# Confirm structure
df_baseline.tail()


,pickup_hour,HB102,lag_1
8742,2024-12-31 19:00:00,8,10.0
8743,2024-12-31 20:00:00,2,8.0
8744,2024-12-31 21:00:00,1,2.0
8745,2024-12-31 22:00:00,0,1.0
8746,2024-12-31 23:00:00,1,0.0


In [14]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Define X and y
X = df_baseline[["lag_1"]]
y = df_baseline[target_location]

# Train-test split (80-20), time-based (no shuffle)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Predict using naive lag (just use lag_1 as prediction)
y_pred = X_test["lag_1"]

# Evaluate MAE
mae = mean_absolute_error(y_test, y_pred)
print("✅ Baseline MAE:", round(mae, 4))


✅ Baseline MAE: 3.6537


In [21]:
# Imports
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load time series data
df = pd.read_csv("../data/ts_data.csv", parse_dates=["pickup_hour"])

# Set target location
target_location = "HB102"  # Change if needed

# Create baseline dataframe with just pickup_hour and target location
df_baseline = df[["pickup_hour", target_location]].copy()

# Create 1-hour lag feature
df_baseline["lag_1"] = df_baseline[target_location].shift(1)

# Drop rows with NaNs caused by shifting
df_baseline = df_baseline.dropna().reset_index(drop=True)

# OPTIONAL: Remove rows where actual target is zero to avoid distorted RMSE impact (optional, only if justified)
# df_baseline = df_baseline[df_baseline[target_location] != 0].copy()

# Define features and target
X = df_baseline[["lag_1"]]
y = df_baseline[target_location]

# Time-based train-test split (no shuffle)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Predict using naive lag
y_pred = X_test["lag_1"]

# Evaluate MAE and RMSE
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Display results
print("✅ Baseline MAE:", round(mae, 4))
print("✅ Baseline RMSE:", round(rmse, 4))


✅ Baseline MAE: 3.6537
✅ Baseline RMSE: 6.9605


In [30]:
import mlflow
import os

# ✅ Set your DagsHub MLflow credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "sai-snehitha"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "546ba070a0d146826c8d49111843d556196bcf9a"  # Replace only if expired later

# ✅ Set the MLflow tracking URI and experiment
mlflow.set_tracking_uri("https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow")
mlflow.set_experiment("citi-bike-project")

# ✅ Log the already computed MAE
with mlflow.start_run(run_name="Baseline Model"):
    mlflow.log_metric("mae", 3.6537)
    print("✅ Baseline MAE logged to DagsHub MLflow.")


2025/05/07 18:02:02 INFO mlflow.tracking.fluent: Experiment with name 'citi-bike-project' does not exist. Creating a new experiment.


✅ Baseline MAE logged to DagsHub MLflow.
🏃 View run Baseline Model at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1/runs/9f170cf4df0f4c609aad9eac5efe56a7
🧪 View experiment at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1


In [32]:
import os
import mlflow
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Set DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "sai-snehitha"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "546ba070a0d146826c8d49111843d556196bcf9a"

# Set MLflow URI and experiment
mlflow.set_tracking_uri("https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow")
mlflow.set_experiment("citi-bike-project")

# Load full preprocessed time series data
df = pd.read_csv("../data/ts_data.csv", parse_dates=["pickup_hour"])

# List of target locations
locations = ["HB102", "HB105", "JC115"]

for location in locations:
    # Create baseline dataframe
    df_baseline = df[["pickup_hour", location]].copy()
    df_baseline["lag_1"] = df_baseline[location].shift(1)
    df_baseline = df_baseline.dropna().reset_index(drop=True)

    # Features & Target
    X = df_baseline[["lag_1"]]
    y = df_baseline[location]

    # Time-based train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Naive prediction
    y_pred = X_test["lag_1"]

    # Compute MAE
    mae = mean_absolute_error(y_test, y_pred)
    print(f"✅ Baseline MAE for {location}: {mae:.4f}")

    # Log to MLflow with tags
    with mlflow.start_run(run_name=f"Baseline - {location}"):
        mlflow.set_tag("model_type", "Baseline")
        mlflow.set_tag("location_id", location)
        mlflow.log_metric("mae", mae)


✅ Baseline MAE for HB102: 3.6537
🏃 View run Baseline - HB102 at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1/runs/c251c45fe82d4df89a59cd59b583c745
🧪 View experiment at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1
✅ Baseline MAE for HB105: 1.8686
🏃 View run Baseline - HB105 at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1/runs/cdb323184f344fc689ccac1eb219161e
🧪 View experiment at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1
✅ Baseline MAE for JC115: 2.8640
🏃 View run Baseline - JC115 at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1/runs/e4c17d18b31047059e4cd5d2420505cd
🧪 View experiment at: https://dagshub.com/sai-snehitha/citi-bike-prediction-system.mlflow/#/experiments/1
